# COVID-19 India Overview
> Tracking coronavirus total cases, deaths and new cases in India

- author: Hargun Oberoi
- image: images/covid-overview.png
- permalink: /covid-overview-linde/
- hide: true
- badges: false

In [1]:
#hide
import pandas as pd
import numpy as np
import requests
import datetime as dt
from jinja2 import Template
from IPython.display import HTML
import json
import requests

In [2]:
#hide
url = 'https://raw.githubusercontent.com/hargun3045/dashboards/master/abb_dict.json'
r = requests.get(url)
abb_dict = r.json()

In [3]:
#hide
#write a function that checks if all last index values are zero, and also selects the appropriate date columns
def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['stateABB','states'])]
    valid_i = -1
    for i in range(-1,-len(dt_cols),-1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            valid_i = i
            break
    return valid_i, dt_cols

In [4]:
#hide
def get_frame(name):
#    pdb.set_trace()
    url = f'http://api.covid19india.org/states_daily_csv/{name}.csv'
    df = pd.read_csv(url)
    df.set_index('date',inplace = True)
    df = df.T
    df = df.iloc[:-1,:]
    df.fillna(0,inplace=True)
    df = df[df.columns].astype(int)
    df = df.rename_axis('stateABB').reset_index()
    df['states'] = df['stateABB'].map(abb_dict)
    df = df[[df.columns[-1]] + list(df.columns[:-1])]
    df =df.iloc[1:,:]
    df = df[df.states.notna()]
    df.sort_values(by = [df.columns[-1]], ascending = False, inplace = True)
    df.drop(['stateABB'],axis = 1)
    df = df.groupby('states').sum().cumsum(axis=1)
    df.reset_index(inplace = True)
    return df

In [5]:
#hide
dft_cases = get_frame('confirmed')
dft_deaths = get_frame('deceased')

In [6]:
#hide
valid_i, dt_cols = get_dates(dft_cases)
dt_today = dt_cols[valid_i]
dt_yday = dt_cols[valid_i -1]

In [7]:
#hide
dfc_cases = dft_cases.groupby('states')[dt_today].sum()
dfc_deaths = dft_deaths.groupby('states')[dt_today].sum()
dfp_cases = dft_cases.groupby('states')[dt_yday].sum()
dfp_deaths = dft_deaths.groupby('states')[dt_yday].sum()

In [8]:
#hide
# Build the cases table
df_table = (pd.DataFrame(dict(Cases = dfc_cases, Deaths = dfc_deaths, PCases = dfp_cases, PDeaths = dfp_deaths))
            .sort_values(by = ['Cases','Deaths'], ascending = [False, False])
            .reset_index())
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)
    #Clip ça veut dire, les chiffres negatif sont interdit
df_table['Fatality Rate'] = (100* df_table['Deaths']/ df_table['Cases']).round(2)

In [9]:
#hide
#Building the summary tab
metrics = [df_table.columns[index] for index in [1,2,5,6]]
s_mah = df_table[df_table.states == 'Maharashtra'][metrics].sum().add_prefix('MH ')
s_del = df_table[df_table.states =='Delhi'][metrics].sum().add_prefix('DEL ')
s_guj = df_table[df_table.states == 'Gujarat'][metrics].sum().add_prefix('GUJ ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_yday)}
summary = {**summary,**df_table[metrics].sum(), **s_mah, **s_del, **s_guj}

In [10]:
#hide
#Data on new cases
dft_ct_new_cases = dft_cases.groupby('states')[dt_cols].sum().diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.sort_values(by = [dft_ct_new_cases.columns[-1]], ascending = False,inplace = True)

In [11]:
#hide
# Get total number from world data only
COL_REGION = 'states'
KPI_CASE = 'India'
kpis_info = [
    {'title': 'Maharashtra', 'prefix': 'MH'},
    {'title': 'Delhi', 'prefix': 'DEL'},
    {'title': 'Gujarat', 'prefix': 'GUJ'}]

In [12]:
#hide
def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

In [13]:
#hide_input
template = Template(get_template('https://raw.githubusercontent.com/hargun3045/dashboards/master/overview.tpl'))
#Local method
#with open('overview.tpl','r') as f:
#     temp = f.read()
#template = Template(temp)
html = template.render(
    D=summary, table=df_table,
    KPI_CASE = KPI_CASE,COL_REGION = COL_REGION,
    KPIS_INFO = kpis_info,# REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[valid_i-14]:dt_cols[valid_i]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Data source: [covid19India](https://www.covid19india.org/)
Thanks to the people running covid19India for sharing their raw data. For more visualisations and essential information please visit their website [www.covid19india.org](https://www.covid19india.org/)